In [1]:
# MedMNIST kütüphanesini indirip sistemimizze yüklüyoruz
! git clone https://github.com/MedMNIST/MedMNIST
! pip install MedMNIST/ && rm -rf MedMNIST
! pip install -r requirements.txt

Cloning into 'MedMNIST'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 156 (delta 84), reused 87 (delta 37), pack-reused 0
Receiving objects: 100% (156/156), 2.29 MiB | 5.72 MiB/s, done.
Resolving deltas: 100% (84/84), done.
Processing ./MedMNIST
  Using cached torch-1.8.0-cp36-cp36m-manylinux1_x86_64.whl (735.5 MB)
  Using cached dataclasses-0.8-py3-none-any.whl (19 kB)
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
  Created wheel for MedMNIST: filename=MedMNIST-0.2.2-py3-none-any.whl size=13970 sha256=e45021879b5281e9c4d8d2b0dfdd80536983aed6b02359115e5de809b3ecd5a7
  Stored in directory: /tmp/pip-ephem-wheel-cache-pi0k0xps/wheels/49/0c/0d/6ea2e235481c92ab5ba6b458810dae2b0aae3ffdb968abf5a7
Successfully built MedMNIST
ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-

Dataset hakkında ayrıntılı bilgi için [MedMNIST](https://medmnist.github.io/)


In [2]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import medmnist
from medmnist.dataset import OrganMNISTAxial
from medmnist.info import INFO
import os

import pandas as pd

from utils import plot_cm, show_random_images, show_train_history, to_one_hot, visualize_sample_data_gif, show_image

In [3]:
# Dataseti indiriyoruz
dataset_name = 'organmnist_axial'
data_path = "data/"
OrganMNISTAxial(root=data_path, transform=None, download=True)

info = INFO[dataset_name]
class_names = info["label"]
n_channels = info['n_channels']
n_classes = len(info['label'])
print(info['description'])
print(f'\nClass Names:')
for idx, c_name in class_names.items():
    print(idx,":", c_name)


OrganMNIST_Axial: A dataset based on 3D computed tomography (CT) images from Liver Tumor Segmentation Benchmark (LiTS). We use bounding-box annotations of 11 body organs from another study to obtain the organ labels. Hounsfield-Unit (HU) of the 3D images are transformed into grey scale with a abdominal window; we then crop 2D images from the center slices of the 3D bounding boxes in axial views (planes). The images are resized into 1 x 28 x 28 to perform multi-class classification of 11 body organs. 115 and 16 CT scans from the source training set are used as training and validation set, respectively. The 70 CT scans from the source test set are treated as the test set.

Class Names:
0 : bladder
1 : femur-left
2 : femur-right
3 : heart
4 : kidney-left
5 : kidney-right
6 : liver
7 : lung-left
8 : lung-right
9 : pancreas
10 : spleen


In [ ]:
# indirdiğimiz npz formatında olan dataseti okuyoruz.
data_npz = np.load(os.path.join(data_path, dataset_name + ".npz"))

data = {}
for key in data_npz.keys():
    data[key] = data_npz[key]
    
train_images, train_labels = data['train_images'], data['train_labels']
val_images, val_labels = data['val_images'], data['val_labels']

train_images.shape, train_labels.shape, val_images.shape, val_labels.shape

In [ ]:
show_random_images(train_images, train_labels, class_names)

In [ ]:
# Originial data
visualize_sample_data_gif()

In [ ]:
# Datasetimizi normalize ediyoruz
min_ = train_images.min()
max_ = train_images.max()

def minmax(images):
    global min_, max_
    return (images - min_) / (max_ - min_)

print(f"Before -> train min: {train_images.min():.2f} train mean: {train_images.mean():.2f} train max: {train_images.max():.2f}")
train_images = minmax(train_images)
val_images = minmax(val_images)

print(f"After  -> train min: {train_images.min():.2f} train mean: {train_images.mean():.2f} train max: {train_images.max():.2f}")

One Hot Encoding
<br>
![title](https://miro.medium.com/max/700/1*ggtP4a5YaRx6l09KQaYOnw.png)
<br>
https://towardsdatascience.com/building-a-one-hot-encoding-layer-with-tensorflow-f907d686bf39

In [ ]:
train_labels[0:5]

In [ ]:
class_names

In [ ]:
to_one_hot(train_labels, num_classes=n_classes)[0:5]

In [ ]:
# ANN modelimizi oluşturuyoruz

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Flatten(input_shape=(28,28)))
ann.add(tf.keras.layers.Dense(256, activation="relu"))
ann.add(tf.keras.layers.Dropout(0.2))
ann.add(tf.keras.layers.Dense(256, activation="relu"))
ann.add(tf.keras.layers.Dropout(0.2))
ann.add(tf.keras.layers.Dense(n_classes, activation="softmax"))

ann.compile(optimizer="adam",
            loss="categorical_crossentropy", metrics = ["accuracy"])

ann.summary()

## Softmax Activation Function

<div>
<img src="https://ljvmiranda921.github.io/assets/png/cs231n-ann/softmax.png" width="800"/>
</div>

https://ljvmiranda921.github.io/notebook/2017/08/13/softmax-and-the-negative-log-likelihood/

In [ ]:
history = ann.fit(train_images, to_one_hot(train_labels, num_classes=n_classes),
                  validation_data=(val_images, to_one_hot(val_labels, num_classes=n_classes)),
                  epochs=20)
show_train_history(history)

In [ ]:
probas = ann.predict(val_images)
val_preds = np.argmax(probas, axis=-1).reshape(-1,1)

In [ ]:
show_random_images(val_images, val_labels, class_names, n=4, preds=val_preds)

In [ ]:
plot_cm(np.squeeze(val_labels), np.squeeze(val_preds), class_names)

In [ ]:
cnn = tf.keras.models.Sequential()

cnn.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding="same", input_shape=(28,28,1)))
cnn.add(tf.keras.layers.MaxPool2D((2, 2)))
cnn.add(tf.keras.layers.Conv2D(64, (3,3), padding="same", activation='relu'))

cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(100, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.2))
cnn.add(tf.keras.layers.Dense(n_classes, activation='softmax'))

cnn.compile(optimizer="adam",
            loss="categorical_crossentropy", metrics = ["accuracy"])

cnn.summary()

In [ ]:
cnn_history = cnn.fit(np.expand_dims(train_images, axis=-1), 
        to_one_hot(train_labels, num_classes=n_classes),
        validation_data=(np.expand_dims(val_images, axis=-1), tf.keras.utils.to_categorical(val_labels, num_classes=n_classes)),
        epochs=20)
show_train_history(cnn_history)


In [ ]:
probas = cnn.predict(np.expand_dims(val_images,axis=-1))
val_preds = np.argmax(probas, axis=-1).reshape(-1,1)

In [ ]:
show_random_images(val_images, val_labels, class_names, n=4, preds=val_preds)

In [ ]:
plot_cm(np.squeeze(val_labels), np.squeeze(val_preds), class_names)

In [ ]:
cnn.evaluate(np.expand_dims(val_images, axis=-1), to_one_hot(val_labels,n_classes))

In [ ]:
cnn.evaluate(np.expand_dims(test_images, axis=-1), to_one_hot(test_labels,n_classes))

In [ ]:
probas = cnn.predict(np.expand_dims(test_images,axis=-1))
val_preds = np.argmax(probas, axis=-1).reshape(-1,1)

In [ ]:
show_random_images(test_images, test_labels, class_names, n=4, preds=val_preds)

In [ ]:
class CNN(tf.keras.Model):

    def __init__(self, num_classes):
        super(CNN, self).__init__()
        
        self.conv1 = tf.keras.layers.Conv2D()
        self.block_2 = ResNetBlock()
        self.global_pool = layers.GlobalAveragePooling2D()
        self.classifier = Dense(num_classes)

    def call(self, inputs):
        x = self.block_1(inputs)
        x = self.block_2(x)
        x = self.global_pool(x)
        return self.classifier(x)
